In [31]:
import findspark
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')
import pyspark

In [32]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Clustering').getOrCreate()

In [33]:
from pyspark.ml.clustering import KMeans

In [34]:
dataset = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [35]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [36]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [37]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [38]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [39]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [40]:
final_data = vec_assembler.transform(dataset)

In [41]:
from pyspark.ml.feature import StandardScaler

In [42]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [43]:
scalerModel = scaler.fit(final_data)

In [44]:
cluster_final_data = scalerModel.transform(final_data)

In [45]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [46]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [47]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [48]:
evaluator=ClusteringEvaluator(predictionCol='prediction',featuresCol='scaledFeatures',metricName='silhouette')

In [49]:
predictions1=model_k3.transform(cluster_final_data)
predictions2=model_k2.transform(cluster_final_data)


In [50]:
sqauredEuclideanDistance1=evaluator.evaluate(predictions1)
sqauredEuclideanDistance2=evaluator.evaluate(predictions2)

In [51]:
sqauredEuclideanDistance1

0.7608455651454915

In [52]:
sqauredEuclideanDistance2

0.817646009401248

In [54]:
centers1=model_k3.clusterCenters()

In [56]:
print (centers1)

[array([2.93719177, 2.88492202, 0.        , 3.19938371, 4.52857793,
       3.30407351]), array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
       5.26676612]), array([3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
       3.26738378])]


In [57]:
centers2=model_k2.clusterCenters()

In [59]:
print(centers2)

[array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
       5.26676612]), array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
       3.28474   ])]


In [61]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   88|
|         0|   79|
+----------+-----+



In [62]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

